In [98]:
import json 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import cv2

In [99]:
path_to_ballet1="C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\TB_S\\TB_S_000000000005_keypoints.json"
path_to_ballet2="C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\TB_S_FB\\TB_S_FB_000000000005_keypoints.json"
path_3="C:\\Users\\user\\Desktop\\project\\1453169-20200914T174236Z-001\\SYN_K\\SYN_K_000000000009_keypoints.json"

In [100]:
f1=open(path_to_ballet1)
f2=open(path_to_ballet2)
f3=open(path_3)

In [101]:
data_f1= json.load(f1)
data_f2= json.load(f2)
data_f3=json.load(f3)

In [102]:
def read_keypoints(data):
    people_column=data['people']
    for d in people_column:
        keypoints=(d["pose_keypoints_2d"])
    return keypoints  


def remove_confidence_interval(data):

    j=2
    keypoints=read_keypoints(data)
    keypoints_woconfidence=keypoints.copy()
    while j<=len(keypoints_woconfidence):
        keypoints_woconfidence.pop(j)
        j+=2
    return keypoints_woconfidence

def remove_zeros_fromall(data1,data2):
    
    
    XY1_vector=remove_confidence_interval(data1)

    XY2_vector=remove_confidence_interval(data2)

    data = {'frame1': XY1_vector, 
            'frame2': XY2_vector}
    df = pd.DataFrame(data)
    df=df[df.frame1!=0]
    df=df[df.frame2!=0] 
    frame1=df['frame1'].to_numpy()
    frame2=df['frame2'].to_numpy()
    return frame1,frame2

In [103]:
data_f1

{'version': 1.3,
 'people': [{'person_id': [-1],
   'pose_keypoints_2d': [214.341,
    63.312,
    0.902298,
    220.31,
    101.339,
    0.95302,
    196.344,
    101.343,
    0.870564,
    164.293,
    131.309,
    0.859177,
    168.338,
    140.317,
    0.702319,
    243.343,
    100.356,
    0.880465,
    266.316,
    120.301,
    0.86968,
    292.302,
    126.31,
    0.705414,
    220.312,
    179.301,
    0.749332,
    205.34,
    179.317,
    0.716647,
    215.315,
    245.341,
    0.746386,
    217.319,
    309.328,
    0.681233,
    234.345,
    179.299,
    0.707742,
    224.341,
    246.323,
    0.745144,
    221.353,
    313.31,
    0.725757,
    211.281,
    59.2967,
    0.946076,
    220.323,
    58.3423,
    0.962496,
    205.317,
    64.3315,
    0.276184,
    230.35,
    63.3083,
    0.886045,
    201.31,
    323.298,
    0.573108,
    206.33,
    324.308,
    0.540061,
    226.339,
    319.329,
    0.681732,
    198.309,
    322.332,
    0.47191,
    198.334,
    319.

In [104]:
remove_zeros_fromall(data_f1,data_f3)

(array([214.341 ,  63.312 , 220.31  , 101.339 , 196.344 , 101.343 ,
        164.293 , 131.309 , 168.338 , 140.317 , 243.343 , 100.356 ,
        266.316 , 120.301 , 292.302 , 126.31  , 220.312 , 179.301 ,
        205.34  , 179.317 , 215.315 , 245.341 , 217.319 , 309.328 ,
        234.345 , 179.299 , 224.341 , 246.323 , 221.353 , 313.31  ,
        211.281 ,  59.2967, 220.323 ,  58.3423, 205.317 ,  64.3315,
        230.35  ,  63.3083, 201.31  , 323.298 , 206.33  , 324.308 ,
        226.339 , 319.329 , 198.309 , 322.332 , 222.334 , 317.302 ]),
 array([240.256, 152.226, 271.273, 164.223, 275.23 , 184.252, 277.266,
        229.249, 276.22 , 269.241, 270.23 , 147.225, 244.27 , 185.28 ,
        215.237, 207.245, 337.262, 205.25 , 326.283, 213.279, 323.255,
        275.24 , 376.242, 271.263, 347.259, 196.274, 389.233, 239.231,
        453.254, 266.265, 235.231, 154.222, 239.261, 145.253, 237.231,
        162.257, 248.254, 139.271, 484.272, 282.242, 485.241, 278.279,
        458.252, 267.243, 38

In [105]:
def similarity_between_2frames(data1,data2):
    XY1_vector,XY2_vector=remove_zeros_fromall(data1,data2)    

    #since they are 1d
    #XY1_vector.reshape(1,-1)
    #XY2_vector.reshape(1,-1)


    similarity_X=cosine_similarity(XY1_vector.reshape(1,-1),XY2_vector.reshape(1,-1))

    print("Similarity for XY between two frames is:", similarity_X)

In [106]:
similarity_between_2frames(data_f1,data_f3)

Similarity for XY between two frames is: [[0.94025011]]


# Similarity per bodypart

Define the bodyparts

In [107]:
def create_y_coordicate(data):
    df=remove_confidence_interval(data)
    df=np.array(df)
    mask = np.ones(df.size, dtype=bool)
    mask[1::2] = 0
    points = df[mask]
    y=points.tolist()
    return y

def create_x_coordicate(data):
    df=remove_confidence_interval(data)
    df=np.array(df)
    mask = np.zeros(df.size, dtype=bool)
    mask[1::2] = 1
    points = df[mask]
    x=points.tolist()
    return x

def bodyparts(data):        
    
    # //     {0,  "Nose"}, 
    # //     {1,  "Neck"},
    # //     {2,  "RShoulder"},
    # //     {3,  "RElbow"},
    # //     {4,  "RWrist"},
    # //     {5,  "LShoulder"},
    # //     {6,  "LElbow"},
    # //     {7,  "LWrist"},
    # //     {8,  "MidHip"},
    # //     {9,  "RHip"},
    # //     {10, "RKnee"},
    # //     {11, "RAnkle"},
    # //     {12, "LHip"},
    # //     {13, "LKnee"},
    # //     {14, "LAnkle"},
    # //     {15, "REye"},
    # //     {16, "LEye"},
    # //     {17, "REar"},
    # //     {18, "LEar"},
    # //     {19, "LBigToe"},
    # //     {20, "LSmallToe"},
    # //     {21, "LHeel"},
    # //     {22, "RBigToe"},
    # //     {23, "RSmallToe"},
    # //     {24, "RHeel"},
    # //     {25, "Background"}    
    
    x=create_x_coordicate(data)
    y=create_y_coordicate(data)
    

    nose_to_neck=(x[0]-x[1]),(y[0]-y[1])
    
    right_eye_to_nose=(x[0]-x[15]),(y[0]-y[15])
    
    right_ear_to_eye=(x[15]-x[17]),(y[15]-y[17])
    left_eye_to_nose=(x[0]-x[16]),(y[0]-y[16])
    left_ear_to_eye=(x[16]-x[18]),(y[16]-y[18])
    
    torso= (x[1]-x[8]),(y[1]-y[8])
    
    
    right_shoulder=(x[1]-x[2]),(y[1]-y[2])
    left_shoulder=(x[1]-x[5]),(y[1]-y[5])
    
    right_upper_arm=(x[2]-x[3]),(y[2]-y[3])
    right_lower_arm=(x[3]-x[4]),(y[3]-y[4])
    left_upper_arm=(x[5]-x[6]),(y[5]-y[6])
    left_lower_arm=(x[6]-x[7]),(y[6]-y[7])
    
    right_hip=(x[8]-x[9]),(y[8]-y[9])
    right_upper_leg=(x[9]-x[10]),(y[9]-y[10])
    right_lower_leg=(x[10]-x[11]),(y[10]-y[11])
    left_hip=(x[8]-x[12]),(y[8]-y[12])
    left_upper_leg=(x[12]-x[13]),(y[12]-y[13])
    left_lower_leg=(x[13]-x[14]),(y[13]-y[14])
    
    left_foot=(x[14]-x[19]),(y[14]-y[19])
    left_toes=(x[19]-x[20]),(y[19]-y[20])
    left_ankle_to_heel=(x[14]-x[21]),(y[14]-y[21]) 
    right_foot=(x[11]-x[22]),(y[11]-y[22])
    right_toes=(x[22]-x[23]),(y[22]-y[23])
    right_ankle_to_heel=(x[11]-x[24]),(y[11]-y[24])
    
    body_vector=np.array([nose_to_neck, left_eye_to_nose ,left_ear_to_eye, right_eye_to_nose,right_ear_to_eye, torso,left_shoulder, right_shoulder,right_upper_arm ,right_lower_arm ,left_upper_arm ,left_lower_arm,left_hip ,left_upper_leg ,left_lower_leg ,right_hip ,right_upper_leg,right_lower_leg,right_foot ,right_toes ,right_ankle_to_heel ,left_foot ,left_toes ,left_ankle_to_heel])
    return body_vector

In [108]:
def similarity_bodyparts(data1,data2):
    #computes the similarity for every stick vector in the figures in one frame. 
    #I compute all the stick vectors per figure and then I do a pairwise comparison 
    #(also with cosine measure). 
    #I take the average of all similarities to come up with an overall similarity.
    bodyvector1=bodyparts(data1)
    bodyvector2=bodyparts(data2)
    cos_vector=np.array([])
    for i in range(len(bodyvector1)):
        a=cosine_similarity(bodyvector1[i].reshape(1,-1),bodyvector2[i].reshape(1,-1))
        #delete arrays with zeros
        if a != (np.array([0])):
            cos_vector=np.append(cos_vector,a )
        
    return round(np.mean(cos_vector),3) 

In [109]:
similarity_bodyparts(data_f2,data_f1)

0.282

Angle Similarity

In [110]:
#produces RunitmeWarning for division with zero
np.seterr(divide='ignore', invalid='ignore')

#for every neighbor vectors compute angles
def compute_angle_in_degrees(vector1,vector2):   
    unit_vector_1 = vector1/ np.linalg.norm(vector1)
    unit_vector_2 = vector2/ np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return degree(angle)


def compute_angle(vector1,vector2):   
    unit_vector_1 = vector1/ np.linalg.norm(vector1)
    unit_vector_2 = vector2/ np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    if np.isnan(angle):
        return 0
    else:
        return math.cos(angle)

In [111]:
#create a vector with all angles
def compute_angle_vector(data):
    x=create_x_coordicate(data)
    y=create_y_coordicate(data)
    
    nose_to_neck=(x[0]-x[1]),(y[0]-y[1])
    
    right_shoulder=(x[1]-x[2]),(y[1]-y[2])
    left_shoulder=(x[1]-x[5]),(y[1]-y[5])
      
    angle_nose_to_neck_to_left_shoulder=compute_angle(nose_to_neck,left_shoulder)
    angle_nose_to_neck_to_right_shoulder=compute_angle(nose_to_neck,right_shoulder)
    angle_left_shoulder_to_right_shoulder=compute_angle(left_shoulder,right_shoulder)
    
    left_upper_arm=(x[5]-x[6]),(y[5]-y[6])
    angle_left_shoulder_to_left_upper_arm=compute_angle(left_upper_arm,left_shoulder)
    
    left_lower_arm=(x[6]-x[7]),(y[6]-y[7])
    angle_left_lower_arm_to_left_upper_arm=compute_angle(left_upper_arm,left_lower_arm)
    
    right_upper_arm=(x[2]-x[3]),(y[2]-y[3])
    angle_right_upper_arm_to_right_shoulder=compute_angle(right_upper_arm,right_shoulder)
    
    right_lower_arm=(x[3]-x[4]),(y[3]-y[4])
    angle_right_upper_arm_to_right_lower_arm=compute_angle(right_upper_arm,right_lower_arm)
    
    left_eye_to_nose=(x[0]-x[16]),(y[0]-y[16])
    left_ear_to_eye=(x[16]-x[18]),(y[16]-y[18])
    
    angle_left_eye_to_nose_to_left_ear_to_eye=compute_angle(left_eye_to_nose,left_ear_to_eye)
    
    angle_left_eye_to_nose_to_neck=compute_angle(left_eye_to_nose,nose_to_neck)

    right_eye_to_nose=(x[0]-x[15]),(y[0]-y[15])
    right_ear_to_eye=(x[15]-x[17]),(y[15]-y[17])

    angle_nose_to_neck_to_right_eye_to_nose=compute_angle(nose_to_neck,right_eye_to_nose)  
    angle_left_eye_to_nose_to_rigt_ear_to_eye=compute_angle(left_eye_to_nose,right_ear_to_eye)
    
    angle_right_eye_to_nose_to_right_ear_to_eye=compute_angle(right_eye_to_nose,right_ear_to_eye)

    right_hip=(x[8]-x[9]),(y[8]-y[9])
    right_upper_leg=(x[9]-x[10]),(y[9]-y[10])
    
    angle_right_hip_to_right_upper_leg=compute_angle(right_hip,right_upper_leg)
    
    right_lower_leg=(x[10]-x[11]),(y[10]-y[11])
        
    angle_right_upper_leg_to_right_lower_leg=compute_angle(right_upper_leg,right_lower_leg)    
    
    left_hip=(x[8]-x[12]),(y[8]-y[12])
    left_upper_leg=(x[12]-x[13]),(y[12]-y[13])
    
    angle_left_hip_to_left_upper_leg=compute_angle(left_hip,left_upper_leg)
    
    left_lower_leg=(x[13]-x[14]),(y[13]-y[14])
    
    angle_left_upper_leg_to_left_lower_leg=compute_angle(left_upper_leg,left_lower_leg)
    
    left_ankle_to_heel=(x[14]-x[21]),(y[14]-y[21]) 
    
    angle_left_lower_leg_left_ankle_to_heel=compute_angle(left_lower_leg,left_ankle_to_heel)
    
    right_ankle_to_heel=(x[11]-x[24]),(y[11]-y[24])
    
    angle_right_lower_leg_to_right_ankle_to_heel=compute_angle(right_lower_leg,right_ankle_to_heel)
    
    right_toes=(x[22]-x[23]),(y[22]-y[23])
    right_foot=(x[11]-x[22]),(y[11]-y[22])
    
    angle_right_foot_to_right_toes=compute_angle(right_foot,right_toes)
    angle_right_foot_to_right_lower_leg=compute_angle(right_foot,right_lower_leg)
    angle_right_foot_to_right_ankle_to_heel=compute_angle(right_foot,right_ankle_to_heel)
    
    left_foot=(x[14]-x[19]),(y[14]-y[19])
    left_toes=(x[19]-x[20]),(y[19]-y[20])
    
    angle_left_foot_to_left_lower_leg=compute_angle(left_foot, left_lower_leg)
    angle_left_foot_to_left_ankle_to_heel=compute_angle(left_foot, left_ankle_to_heel)
    angle_left_foot_to_left_toes=compute_angle(left_foot,left_toes)
    
    torso= (x[1]-x[8]),(y[1]-y[8])
    
    angle_torso_to_right_shoulder=compute_angle(torso,right_shoulder)
    angle_torso_to_left_shoulder=compute_angle(torso,left_shoulder)
    angle_torso_to_nose_to_neck=compute_angle(torso,nose_to_neck)
    
    angle_torso_to_right_hip=compute_angle(torso,right_hip)
    angle_torso_to_left_hip=compute_angle(torso,left_hip)
    
    
    body_vector=np.array([angle_nose_to_neck_to_left_shoulder,angle_nose_to_neck_to_right_shoulder,angle_left_shoulder_to_right_shoulder, angle_left_shoulder_to_left_upper_arm,angle_left_lower_arm_to_left_upper_arm,angle_right_upper_arm_to_right_shoulder,angle_right_upper_arm_to_right_lower_arm, angle_left_eye_to_nose_to_left_ear_to_eye, angle_left_eye_to_nose_to_neck,angle_nose_to_neck_to_right_eye_to_nose, angle_left_eye_to_nose_to_rigt_ear_to_eye, angle_right_eye_to_nose_to_right_ear_to_eye,angle_right_hip_to_right_upper_leg,angle_right_upper_leg_to_right_lower_leg, angle_left_hip_to_left_upper_leg,angle_left_upper_leg_to_left_lower_leg,angle_left_lower_leg_left_ankle_to_heel, angle_right_lower_leg_to_right_ankle_to_heel, angle_right_foot_to_right_toes, angle_right_foot_to_right_lower_leg, angle_right_foot_to_right_ankle_to_heel, angle_left_foot_to_left_lower_leg,  angle_left_foot_to_left_ankle_to_heel,  angle_left_foot_to_left_toes ,angle_torso_to_right_shoulder ,angle_torso_to_left_shoulder , angle_torso_to_nose_to_neck, angle_torso_to_right_hip ,angle_torso_to_left_hip  ])
    #print(body_vector)
    return body_vector

In [112]:
compute_angle_vector(data_f2)

array([-0.90406357,  0.93943514, -0.99578982,  0.95215565,  0.9757783 ,
        0.63630612,  0.15714254,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.10809707,  0.99919501,  0.00477033,
        0.99066041,  0.83390169,  0.70450453, -0.96440893,  0.62167101,
       -0.11792229,  0.04800248, -0.51124743, -0.75234669,  0.10634576,
       -0.01475191,  0.44068817,  0.01228135,  0.05281341])

In [113]:
np.isnan(compute_angle_vector(data_f2)[8])

False

In [114]:
from scipy.spatial import distance
from scipy.spatial.distance import cdist
    
def similarity_bodyparts_with_angles(data1,data2):
    bodyvector1=compute_angle_vector(data1)
    bodyvector2=compute_angle_vector(data2)
    
    bodyvector1= bodyvector1.reshape(1,-1)
    bodyvector2= bodyvector2.reshape(1,-1)
    
    return cosine_similarity(bodyvector1, bodyvector2)

In [115]:
similarity_bodyparts_with_angles(data_f2,data_f1)

array([[0.75114975]])